# Demo project

In [1]:
import folium
import pandas as pd
import numpy as np
from ipywidgets import interact
import matplotlib.pyplot as plt

In [2]:
# разбор прогнозов на дату
def date_parser(x):
    if int(x.split('_')[2]) < 10:
        date = pd.to_datetime(x.split('_')[1] + ' 0' + x.split('_')[2])
    else:
        date = pd.to_datetime(x.split('_')[1] + ' ' + x.split('_')[2])
    return date

In [19]:
#Прорисовка интерактивной карты с управляющими элементами
def interact_map(day, hour, predictions, real_data):
    f = folium.Figure(width=750, height=750)
    
    m = folium.Map(location=[lat1, lng1], zoom_start=11, min_zoom=9).add_to(f)
    
    base_map = folium.FeatureGroup(name='Basemap', overlay=True, control=False)
    folium.TileLayer(tiles='OpenStreetMap').add_to(base_map)
    base_map.add_to(m)
    
    heat_map = real_data[real_data.time == day]
    mask1 = predictions.date == day
    mask2 = predictions.hour == hour
    predict_hours = predictions.predict_time.unique()
    
    for i in predict_hours:
        mask3 = predictions.predict_time == i
        heat_map = predictions[mask1 & mask2 & mask3]
        CH=folium.features.Choropleth(geo_data=regs_json,
                                   data=heat_map,
                                   columns=['region','rides'],
                                   key_on="feature.region",
                                   line_weight=0,
                                   legend_name=f'{i} hour prediction',
                                   fill_opacity=0.85,
                                   fill_color='YlOrBr',
                                   nan_fill_opacity=0,
                                   name=f'{i} hour prediction',
                                   show=False,
                                   overlay=False,
                                   bins=6, highlight=True)
        
        #убрать легенды с карты, мешают
        for child in CH._children:
            if child.startswith('color_map'):
                del(CH._children[child])
    
        CH.add_to(m)
 
    folium.LayerControl().add_to(m)
    return f

In [15]:
#рисовка интерактивного графика
def interact_graph(region, pred_hour, predictions, june_real_data):
    T = predictions[(predictions.region==region) & (predictions.predict_time==pred_hour)].date_hour + pd.Timedelta(hours=pred_hour-1)
    X = predictions[(predictions.region==region) & (predictions.predict_time==pred_hour)].rides
    plt.figure(figsize=(20,5))
    plt.ion()
    plt.plot(T, X)
    plt.plot(june_real_data.loc[:, 'time'],june_real_data.loc[:, region])
    plt.legend([f'{region} predictions at {pred_hour}-hour model',f'{region} real june rides'], prop={'size': 16})
    plt.show()

Загрузка и корректировка данных

In [5]:
june_real_data = pd.read_csv('aggr_06_2016.csv', index_col=0, parse_dates=True).T
regions = pd.read_csv('regions.csv', delimiter=';')
june_pred_data = pd.read_csv('week6_final_rf.csv')

In [6]:
june_real_data.reset_index(inplace=True)
june_real_data.rename(columns={'index':'time'}, inplace=True)
june_real_data['time']= pd.to_datetime(june_real_data['time'])
june_real_data.head()

region,time,1,2,3,4,5,6,7,8,9,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
0,2016-06-01 00:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2016-06-01 01:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2016-06-01 02:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2016-06-01 03:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2016-06-01 04:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
june_pred_data.head()

,id,y
0,1075_2016-05-31_23_1,24
1,1075_2016-05-31_23_2,15
2,1075_2016-05-31_23_3,6
3,1075_2016-05-31_23_4,3
4,1075_2016-05-31_23_5,2


In [8]:
%%time
#в отдельный ДФ таблицу предсказаний
predictions = pd.DataFrame()
predictions['date'] = june_pred_data.iloc[:, 0].apply(lambda x: pd.to_datetime(x.split('_')[1]))
predictions['hour'] = june_pred_data.iloc[:, 0].apply(lambda x: int(x.split('_')[2]))
predictions['region'] = june_pred_data.iloc[:, 0].apply(lambda x: int(x.split('_')[0]))
predictions['predict_time'] = june_pred_data.iloc[:, 0].apply(lambda x: int(x.split('_')[-1]))
predictions['rides'] = june_pred_data['y']
predictions['date_hour'] = june_pred_data.iloc[:, 0].apply(date_parser)

Wall time: 1min 8s


In [9]:
predictions.head()

,date,hour,region,predict_time,rides,date_hour
0,2016-05-31,23,1075,1,24,2016-05-31 23:00:00
1,2016-05-31,23,1075,2,15,2016-05-31 23:00:00
2,2016-05-31,23,1075,3,6,2016-05-31 23:00:00
3,2016-05-31,23,1075,4,3,2016-05-31 23:00:00
4,2016-05-31,23,1075,5,2,2016-05-31 23:00:00


In [10]:
#стартовые точки для генерации карты
lng1 = -73.985428
lat1 = 40.748817

Регионы для считывания в GeoSon

In [11]:
regs_json = {'type': "FeatureCollection", 'features': 
    [{'type': 'Feature', 'region': elem[0], 
     'geometry': {'type': 'Polygon', "coordinates": [
             [ [elem[2],elem[3]], [elem[1], elem[3]], [elem[1], elem[4]],[elem[2], elem[4]] ]
             ]},
     'properties': {'region': elem[0]}
    } for elem in regions.itertuples(index=False)]
    }

Интерактивная карта. При выборе даты/часа перегружается. Слои показывают плотность спроса по прогнозам на 1-6 часов вперед

In [20]:
interact(lambda day, hour: interact_map(day, hour, predictions, june_real_data),
         day=pd.date_range(start='2016-06-01', end='2016-06-30').strftime('%Y-%m-%d'),
         hour=np.sort(predictions.hour.unique()))

interactive(children=(Dropdown(description='day', options=('2016-06-01', '2016-06-02', '2016-06-03', '2016-06-…

<function __main__.<lambda>(day, hour)>

График сравнения предсказаний с реальными значениями спроса по регионам

In [16]:
#добавление кнопок zoom не осилила. 
interact(lambda region, pred_hour: interact_graph(region, pred_hour, predictions, june_real_data),
        region=predictions.region.unique(),
        pred_hour=predictions.predict_time.unique())

interactive(children=(Dropdown(description='region', options=(1075, 1076, 1077, 1125, 1126, 1127, 1128, 1129, …

<function __main__.<lambda>(region, pred_hour)>